# Quasi-dynamic spring slider with 1 state variable

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

### Parameters

In [ ]:
### Rate-and-state
# reference friction coefficient
fr = 0.6 
# reference slip velocity (m/s)
Vr = 1e-6
# direct effect parameter 
a = 0.015
#a = 0.020
# state evolution parameter
b = 0.018
# state evolution distance (m)
dc = 1e-2

### Elasticity
# density (g/cm^3)
rho = 2.54
# shear velocity (m/s)
c = 2.5
# radiation damping (MPa*s/m)
eta = rho*c/2
# normal stress (MPa)
sigma = 50
# initial shear stress (MPa)
tau0 = 0.5793*sigma
# calculate critical spring constant (MPa/m)
kcr = sigma*abs(b-a)/dc
# spring constante (MPa/m)
k = 2*kcr
# loading: shear stress increases at constant rate in 'absence' of slip
oneyear = np.pi*1e7
# delta tau (MPa/s)
dtau = 10/oneyear




In [ ]:
def nullclines(x):
    # x is the non-dimensional varaible x and is a 1D array here
    # xnull and ynull are y values corresponding to x-nullcline and y-nullcline for these nondimensional variables
    xnull = (a-b)/fr*x + 1/(eps2*eps4*fr)-eps1*np.exp(-x)/(eps2*eps4*fr)+1
    ynull = (a-b)/fr*x - eps3*np.exp(-x)/(eps4*fr)+eps1*eps3/(eps4*fr)*np.exp(-2*x)+1
    
    return xnull, ynull

def jac(x,y):
    '''
    x: x coordinate of the fixed point
    y: y coordinate of the fixed point
    '''
    bigBracketX = eps1*np.exp(-x)-1-eps2*eps4*(fr*(1-y)+x*(a-b))
    bigBracketY = eps1*eps3*np.exp(-x)-eps3+np.exp(x)*eps4*(fr*(1-y)+x*(a-b))
    
    Xx = -(1+eps2*eps3*(np.exp(-x)))**(-2)*(-eps2*eps3*np.exp(-x))*bigBracketX + \
         (1+eps2*eps3*np.exp(-x))**(-1)*(-eps1*np.exp(-x)-eps2*eps4*(a-b))
        
    Xy = (1+eps2*eps3*np.exp(-x))**(-1) *eps2*eps4*fr
    
    Yx = -(1+eps2*eps3*(np.exp(-x)))**(-2)*(-eps2*eps3*np.exp(-x))*bigBracketY + \
         (1+eps2*eps3*np.exp(-x))**(-1)* \
         (-eps1*eps3*np.exp(-x)+np.exp(x)*eps4*(fr*(1-y)+x*(a-b))+np.exp(x)*eps4*(a-b))
            
    Yy = -(1+eps2*eps3*np.exp(-x))**(-1)*np.exp(x)*eps4*fr

    
    return np.array([[Xx, Xy],
                     [Yx, Yy]])

#### bifurcation values for sigma

In [ ]:
#### calculate bifurcation values with radiation damping
B = sigma/dc * (a-b)
C = dtau*eta/dc

kcr1 = (-B+np.sqrt(B**2-4*C))/2
kcr2 = (-B-np.sqrt(B**2-4*C))/2

# list of parameters to test
#p_list = np.linspace(0.1*kcr1,10*kcr1,num=1001)

In [ ]:
k = 0.1*kcr1

In [ ]:
Ncr = dc*(-k**2-dtau*eta/dc)/((a-b)*k)

In [ ]:
# list for sigma
p_list = np.linspace(0.01,1000,num=100001)


In [ ]:
eigvals =[]
trace =[]
determinant = []

for i in range(len(p_list)):   
    sigma= p_list[i]

    ### Non-dimensional groups (combos of above)
    eps1 = dtau/(k*Vr) 
    eps2 = sigma*fr/(eta*Vr)
    eps3 = a/fr
    eps4 = eta*Vr/(k*dc*fr)


    ### solve for fixed point
    aa = 1/eps2
    bb = eps3-eps1/eps2
    cc = eps1*eps3

    fp_x1 = np.log((-bb+np.sqrt(bb**2+4*aa*cc))/(2*aa))
    fp_x2 = np.log((-bb-np.sqrt(bb**2+4*aa*cc))/(2*aa))
#     print("Fixed point x-coordinate: ")
#     print(fp_x1)
#     print(fp_x2)
#     print(" ")
    if np.isnan(fp_x1):
        fp_x = fp_x2
    else:
        fp_x = fp_x1

    fp_y = nullclines(fp_x)[0]

    ### Jacobian at the fixed point
    jac_fp = jac(fp_x,fp_y)
#     print("Jacobian: ")
#     print(jac_fp)
#     print(" ")

    ### Eigenvalues of Jacobian at the fixed point
    jac_fp_eigenvals = np.linalg.eigvals(jac_fp)
#     print("Jacobian eigenvals: ")
#     print(jac_fp_eigenvals)
#     print(" ")
    eigvals.append(jac_fp_eigenvals)

    ### Trace of Jacobian at the fixed point
    jac_fp_trace = np.matrix.trace(jac_fp)
#     print("Jacobian trace: ")
#     print(jac_fp_trace)
    trace.append(jac_fp_trace )

    ### Determinant of Jacobian at the fixed point
    jac_fp_det = np.linalg.det(jac_fp)
    determinant.append(jac_fp_det)
    

In [ ]:
eigvals_real = np.real(eigvals)
eigvals_imag = np.imag(eigvals)

## Plot for sigma

In [ ]:
formatter = ScalarFormatter(useMathText=True)

In [ ]:
plt.rcParams.update({'font.size': 16})
ax = plt.figure().add_subplot()
ax.plot(np.log10(p_list),eigvals_real[:,0],'b',linewidth=5)
ax.plot(np.log10(p_list),eigvals_real[:,1],'r',linewidth=3)
ax.axhline(y=0, color='k', linestyle='-', linewidth=2)
ax.xaxis.set_major_formatter(formatter)
ax.yaxis.set_major_formatter(formatter)
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax.set(ylim=(-0.0002, 0.00001))
ax.set(xlabel=r'$log_{10} (\sigma) $',ylabel='Real parts of eigenvalues')


## 

In [ ]:
plt.rcParams.update({'font.size': 16})
ax = plt.figure().add_subplot()
ax.plot(eigvals_real[:,0],eigvals_imag[:,0],'b',linewidth=3)
ax.plot(eigvals_real[:,1],eigvals_imag[:,1],'r',linewidth=3)
ax.xaxis.set_major_formatter(formatter)
ax.yaxis.set_major_formatter(formatter)
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax.set(xlim=(-0.0002, 0.00001))
ax.set(xlabel='Real parts of eigenvalues',ylabel='Imaginary parts of eigenvalues')

## Bifurcation values for k

In [ ]:
# list for sigma
p_list = np.linspace(0.001*kcr1,10000*kcr1,num=100001)


In [ ]:
eigvals =[]
trace =[]
determinant = []

for i in range(len(p_list)):   
    k= p_list[i]

    ### Non-dimensional groups (combos of above)
    eps1 = dtau/(k*Vr) 
    eps2 = sigma*fr/(eta*Vr)
    eps3 = a/fr
    eps4 = eta*Vr/(k*dc*fr)



    ### solve for fixed point
    aa = 1/eps2
    bb = eps3-eps1/eps2
    cc = eps1*eps3

    fp_x1 = np.log((-bb+np.sqrt(bb**2+4*aa*cc))/(2*aa))
    fp_x2 = np.log((-bb-np.sqrt(bb**2+4*aa*cc))/(2*aa))
#     print("Fixed point x-coordinate: ")
#     print(fp_x1)
#     print(fp_x2)
#     print(" ")
    if np.isnan(fp_x1):
        fp_x = fp_x2
    else:
        fp_x = fp_x1

    fp_y = nullclines(fp_x)[0]

    ### Jacobian at the fixed point
    jac_fp = jac(fp_x,fp_y)
#     print("Jacobian: ")
#     print(jac_fp)
#     print(" ")

    ### Eigenvalues of Jacobian at the fixed point
    jac_fp_eigenvals = np.linalg.eigvals(jac_fp)
#     print("Jacobian eigenvals: ")
#     print(jac_fp_eigenvals)
#     print(" ")
    eigvals.append(jac_fp_eigenvals)

    ### Trace of Jacobian at the fixed point
    jac_fp_trace = np.matrix.trace(jac_fp)
#     print("Jacobian trace: ")
#     print(jac_fp_trace)
    trace.append(jac_fp_trace )

    ### Determinant of Jacobian at the fixed point
    jac_fp_det = np.linalg.det(jac_fp)
    determinant.append(jac_fp_det)

In [ ]:
eigvals_real = np.real(eigvals)
eigvals_imag = np.imag(eigvals)

### plot for k

In [ ]:
er0_k = np.sign(eigvals_real[:,0])*np.log10(abs(eigvals_real[:,0]))
er1_k = np.sign(eigvals_real[:,1])*np.log10(abs(eigvals_real[:,1]))

In [ ]:
ei0_k = np.sign(eigvals_imag[:,0])*np.log10(abs(eigvals_imag[:,0]))
ei1_k = np.sign(eigvals_imag[:,1])*np.log10(abs(eigvals_imag[:,1]))

In [ ]:
plt.rcParams.update({'font.size': 16})
ax = plt.figure().add_subplot()
ax.plot(np.log10(p_list),eigvals_real[:,0],'b',linewidth=5)
ax.plot(np.log10(p_list),eigvals_real[:,1],'r',linewidth=3)
ax.axhline(y=0, color='k', linestyle='-', linewidth=2)
ax.xaxis.set_major_formatter(formatter)
ax.yaxis.set_major_formatter(formatter)
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax.set(ylim=(-0.0002, 0.0001))
ax.set(xlabel=r'$log_{10} (k) $',ylabel='Real parts of eigenvalues')

In [ ]:
plt.rcParams.update({'font.size': 16})
ax = plt.figure().add_subplot()
ax.plot(er0_k,ei0_k,'b',linewidth=3)
ax.plot(er1_k,ei1_k,'r',linewidth=3)
ax.xaxis.set_major_formatter(formatter)
ax.yaxis.set_major_formatter(formatter)
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
#ax.set(xlim=(-2e-12, 2e-12))
ax.set(xlabel='Real parts of eigenvalues',ylabel='Imaginary parts of eigenvalues')

In [ ]:
eigvals_real[:,0]

In [ ]:
p_list

In [ ]:
np.sign(eigvals_real[:,0])*np.log10(abs(eigvals_real[:,0]))

In [ ]:
eigvals_imag[:,0]